# Claude Sonnet Todo Grooming Proof of Concept

This notebook demonstrates using Claude Sonnet to groom and enhance todo text statements.

## Setup and Configuration

In [1]:
import requests
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("Environment loaded successfully!")

Environment loaded successfully!


In [2]:
# Claude API configuration
CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
CLAUDE_API_URL = 'https://api.anthropic.com/v1/messages'

if not CLAUDE_API_KEY:
    print("⚠️  Please set CLAUDE_API_KEY environment variable")
    print("Get your API key from: https://console.anthropic.com/")
else:
    print("✓ Claude API key configured")

# Headers for Claude API
headers = {
    'Content-Type': 'application/json',
    'x-api-key': CLAUDE_API_KEY,
    'anthropic-version': '2023-06-01'
}

✓ Claude API key configured


## Todo Grooming Function

In [ ]:
def groom_todo_with_claude(todo_text, context=""):
    """
    Uses Claude Sonnet to groom and enhance a todo text statement from a user.
    
    Args:
        todo_text (str): The original todo text to be groomed
        context (str): Additional context about the project or task
    
    Returns:
        dict: Contains original text, groomed text, and analysis
    """
    
    if not CLAUDE_API_KEY:
        return {"error": "Claude API key not configured"}
    
    prompt = f"""
You are a personal assistant. Your client will give you a text expressing things they must get done. Your task is to understand what is overwhelming the user, breakdown big tasks in smaller tasks and add them to the list. Identify individual tasks and suggest realistic time intervals in which each task could be done. Reword each task and make them more actionable and specific, no fluff, no emojis.

Original todo: "{todo_text}"

Additional context: {context if context else "No additional context provided"}

Please provide:
1. A list of actionable tasks derived from the original todo in the form of a JSON array. each object in the array should have:
   - "task": the reworded task
   - "task_id": a unique identifier for the task, which should be a hexadecimal string of 4 bytes
   - "time_estimate": a realistic time estimate for completion (in hh:mm format). time estimates should be realistic and not too short
   - "dependencies": an array of tasks_id that are blocked if this one is not done yet (if any)
   - "priority": a priority level (low, medium, high)
2. A brief analysis of the original todo text, explaining the breakdown and reasoning behind the tasks. as concise as possible.

Format your response as a JSON.
"""
    
    payload = {
        "model": "claude-3-5-sonnet-20241022",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    
    try:
        response = requests.post(CLAUDE_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        
        result = response.json()
        groomed_content = result['content'][0]['text']
        
        return {
            "success": True,
            "original_todo": todo_text,
            "groomed_analysis": groomed_content,
            "context": context
        }
        
    except requests.exceptions.RequestException as e:
        return {
            "success": False,
            "error": f"API request failed: {str(e)}",
            "original_todo": todo_text
        }
    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error: {str(e)}",
            "original_todo": todo_text
        }

## Test Cases

In [4]:
# Test Case 1: Basic todo statement
print("=" * 60)
print("TEST CASE 1: Basic Todo Statement")
print("=" * 60)

todo1 = "wake up, tell my gramma to take her meds, call my mom, cook and eat breakfast, dont forget to brush my teeth, and get ready for work. get to the bus stop by 8:30am, and then get to work by 9:00am. I need to finish the report by 10:00am, and then have a meeting with the team at 11:00am. After that, I need to review the project plan and send feedback by 1:00pm. Finally, I need to prepare for the presentation tomorrow. I should make time for eat a snack and take a break in between tasks."
context1 = "I need help with my morning routine. I get stressed with big tasks"

result1 = groom_todo_with_claude(todo1, context1)

if result1.get('success'):
    print(f"Original: {result1['original_todo']}")
    print(f"Context: {result1['context']}")
    print("\nGroomed Analysis:")
    print(result1['groomed_analysis'])
else:
    print(f"Error: {result1.get('error')}")

TEST CASE 1: Basic Todo Statement
Original: wake up, tell my gramma to take her meds, call my mom, cook and eat breakfast, dont forget to brush my teeth, and get ready for work. get to the bus stop by 8:30am, and then get to work by 9:00am. I need to finish the report by 10:00am, and then have a meeting with the team at 11:00am. After that, I need to review the project plan and send feedback by 1:00pm. Finally, I need to prepare for the presentation tomorrow. I should make time for eat a snack and take a break in between tasks.
Context: I need help with my morning routine. I get stressed with big tasks

Groomed Analysis:
{
  "analysis": "The todo list shows a morning routine followed by work tasks. The main stressors appear to be time-sensitive work deliverables and managing personal care alongside professional responsibilities. I've broken down the morning routine into smaller, specific actions and separated work tasks with clear time allocations. Built in buffer times for transitions

In [5]:
# Test Case 2: Vague todo statement

In [6]:
# Test Case 3: Complex technical todo

## Interactive Todo Grooming

In [7]:
# Interactive cell - modify these variables and run the cell

## Batch Processing Multiple Todos

In [8]:
# Batch process multiple todos
# todos_batch = [
#     {"todo": "Update documentation", "context": "Python API project"},
#     {"todo": "Add tests", "context": "Django web application"},
#     {"todo": "Optimize database queries", "context": "PostgreSQL database with slow queries"},
#     {"todo": "Implement caching", "context": "High-traffic web application"}
# ]

# print("=" * 60)
# print("BATCH PROCESSING TODOS")
# print("=" * 60)

# for i, item in enumerate(todos_batch, 1):
#     print(f"\n--- Processing Todo {i} ---")
#     result = groom_todo_with_claude(item["todo"], item["context"])
    
#     if result.get('success'):
#         print(f"Original: {result['original_todo']}")
#         print("Groomed version (first 200 chars):")
#         groomed_preview = result['groomed_analysis'][:200] + "..." if len(result['groomed_analysis']) > 200 else result['groomed_analysis']
#         print(groomed_preview)
#     else:
#         print(f"Error: {result.get('error')}")
    
#     print("-" * 40)

## Summary and Next Steps

This proof of concept demonstrates how Claude Sonnet can be used to:

1. **Transform vague todos** into specific, actionable tasks
2. **Add structure** with priority, effort estimates, and acceptance criteria
3. **Identify dependencies** and potential blockers
4. **Break down complex tasks** into manageable sub-tasks
5. **Provide context-aware suggestions** based on the technology stack

### Potential Integrations:
- **Project Management Tools**: Jira, Trello, Asana
- **Development Workflows**: GitHub Issues, GitLab Issues
- **Team Communication**: Slack, Microsoft Teams
- **Task Management**: Todoist, Any.do
